In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s4e11/sample_submission.csv
/kaggle/input/playground-series-s4e11/train.csv
/kaggle/input/playground-series-s4e11/test.csv


# Libraries

In [2]:
# Core Libraries
import pandas as pd
import numpy as np
from scipy import stats
import random
import warnings

# Visualization Libraries
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
import seaborn as sns
import squarify
import plotly.graph_objects as go
import plotly.express as px
%matplotlib inline

# Machine Learning Libraries
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OrdinalEncoder, FunctionTransformer
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.ensemble import IsolationForest
from xgboost import XGBClassifier


# GPU

In [3]:
import torch

def check_gpu_status():
    if torch.cuda.is_available():
        print(f"可用GPU数量: {torch.cuda.device_count()}")
        for i in range(torch.cuda.device_count()):
            print(f"GPU {i}: {torch.cuda.get_device_name(i)}")
            print(f"显存使用情况: {torch.cuda.memory_allocated(i) / 1024**2:.2f} MB 已用, {torch.cuda.memory_reserved(i) / 1024**2:.2f} MB 总分配")
            print(f"显存空余: {torch.cuda.memory_reserved(i) - torch.cuda.memory_allocated(i):.2f} 字节\n")
    else:
        print("未检测到 GPU！")

check_gpu_status()


可用GPU数量: 1
GPU 0: Tesla P100-PCIE-16GB
显存使用情况: 0.00 MB 已用, 0.00 MB 总分配
显存空余: 0.00 字节



# import data

In [4]:
df_train = pd.read_csv("/kaggle/input/playground-series-s4e11/train.csv")
df_test = pd.read_csv("/kaggle/input/playground-series-s4e11/test.csv")

In [5]:
df_train.head()

,id,Name,Gender,Age,City,Working Professional or Student,Profession,Academic Pressure,Work Pressure,CGPA,Study Satisfaction,Job Satisfaction,Sleep Duration,Dietary Habits,Degree,Have you ever had suicidal thoughts ?,Work/Study Hours,Financial Stress,Family History of Mental Illness,Depression
0,0,Aaradhya,Female,49.0,Ludhiana,Working Professional,Chef,NaN,5.0,NaN,NaN,2.0,More than 8 hours,Healthy,BHM,No,1.0,2.0,No,0
1,1,Vivan,Male,26.0,Varanasi,Working Professional,Teacher,NaN,4.0,NaN,NaN,3.0,Less than 5 hours,Unhealthy,LLB,Yes,7.0,3.0,No,1
2,2,Yuvraj,Male,33.0,Visakhapatnam,Student,NaN,5.0,NaN,8.97,2.0,NaN,5-6 hours,Healthy,B.Pharm,Yes,3.0,1.0,No,1
3,3,Yuvraj,Male,22.0,Mumbai,Working Professional,Teacher,NaN,5.0,NaN,NaN,1.0,Less than 5 hours,Moderate,BBA,Yes,10.0,1.0,Yes,1
4,4,Rhea,Female,30.0,Kanpur,Working Professional,Business Analyst,NaN,1.0,NaN,NaN,1.0,5-6 hours,Unhealthy,BBA,Yes,9.0,4.0,Yes,0


# DATA processing

In [6]:
# Drop 'id' column
df_train = df_train.drop(['id'], axis=1)

# Define the target column
target_column = 'Depression'

# Select categorical columns
categorical_columns = df_train.select_dtypes(include=['object']).columns

# Select numerical columns, excluding the target column 
numerical_columns = df_train.select_dtypes(exclude=['object']).columns.drop(target_column)

# Print out the lists of columns
print("Target Column:", target_column)
print("\nCategorical Columns:", categorical_columns.tolist())
print("\nNumerical Columns:", numerical_columns.tolist())

Target Column: Depression

Categorical Columns: ['Name', 'Gender', 'City', 'Working Professional or Student', 'Profession', 'Sleep Duration', 'Dietary Habits', 'Degree', 'Have you ever had suicidal thoughts ?', 'Family History of Mental Illness']

Numerical Columns: ['Age', 'Academic Pressure', 'Work Pressure', 'CGPA', 'Study Satisfaction', 'Job Satisfaction', 'Work/Study Hours', 'Financial Stress']


In [7]:
# Define features and target
X_train = df_train.drop('Depression', axis=1)
y_train = df_train['Depression']

# Define preprocessing pipelines
numerical_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')), 
    ('scaler', StandardScaler()),
    ('convert_to_float32', FunctionTransformer(lambda x: x.astype(np.float32)))
])

categorical_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')), 
    ('ordinal', OrdinalEncoder(dtype=np.int32, handle_unknown='use_encoded_value', unknown_value=-1))
])

# Combine the numerical and categorical pipelines
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_pipeline, numerical_columns),
        ('cat', categorical_pipeline, categorical_columns)
    ]
)

# Apply the transformations to the training and test sets
X_train_preprocessed = preprocessor.fit_transform(X_train)
X_test_preprocessed = preprocessor.transform(df_test) 

# Apply Isolation Forest for outlier detection on the training data
isolation_forest = IsolationForest(contamination=0.04, random_state=10)
outlier_labels = isolation_forest.fit_predict(X_train_preprocessed)

# Filter out outliers from both X_train_preprocessed and y_train
non_outliers_mask = outlier_labels != -1
X_train_preprocessed = X_train_preprocessed[non_outliers_mask]
y_train = y_train[non_outliers_mask]

# XGBoost

In [8]:
import optuna
from sklearn.model_selection import cross_val_score
from xgboost import XGBClassifier

# 定义目标函数
def objective(trial):
    # 定义要优化的超参数
    param_grid = {
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.1, 1.0),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'max_depth': trial.suggest_int('max_depth', 3, 15),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'n_estimators': trial.suggest_int('n_estimators', 100, 500),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'gamma': trial.suggest_float('gamma', 0.0, 5.0),
        'reg_lambda': trial.suggest_float('reg_lambda', 1.0, 10.0),
    }

    # 初始化模型
    model = XGBClassifier(**param_grid, use_label_encoder=False, random_state=10)

    # 交叉验证
    cv_scores = cross_val_score(model, X_train_preprocessed, y_train, cv=5, scoring='accuracy')
    mean_cv_score = cv_scores.mean()

    # 返回目标值（越大越好）
    return mean_cv_score

# 创建 Optuna study
study = optuna.create_study(direction='maximize')  # 最大化准确率
study.optimize(objective, n_trials=50)  # 运行 50 次试验

# 输出最优参数
print("Best parameters:", study.best_params)
print("Best cross-validation accuracy:", study.best_value)

# 使用最优参数初始化模型
best_model_XGB = XGBClassifier(**study.best_params, use_label_encoder=False, random_state=10)
best_model_XGB.fit(X_train_preprocessed, y_train)  # 在整个训练集上训练模型


[I 2024-11-07 16:24:12,080] A new study created in memory with name: no-name-840ec2d1-1b81-47a6-b605-defabcbe3144
[I 2024-11-07 16:24:22,321] Trial 0 finished with value: 0.9433191218526389 and parameters: {'colsample_bytree': 0.28592186403171416, 'learning_rate': 0.0901067554035748, 'max_depth': 10, 'min_child_weight': 3, 'n_estimators': 341, 'subsample': 0.5253760222443411, 'gamma': 3.3343225065131605, 'reg_lambda': 6.570636333244}. Best is trial 0 with value: 0.9433191218526389.
[I 2024-11-07 16:24:28,588] Trial 1 finished with value: 0.9418532363878308 and parameters: {'colsample_bytree': 0.957255814686989, 'learning_rate': 0.24511158389296797, 'max_depth': 8, 'min_child_weight': 3, 'n_estimators': 260, 'subsample': 0.5402264712825626, 'gamma': 4.938503375370101, 'reg_lambda': 4.282768625131128}. Best is trial 0 with value: 0.9433191218526389.
[I 2024-11-07 16:24:38,190] Trial 2 finished with value: 0.942845300866815 and parameters: {'colsample_bytree': 0.5334399036082307, 'learnin

Best parameters: {'colsample_bytree': 0.1588774139080938, 'learning_rate': 0.12142782379625516, 'max_depth': 12, 'min_child_weight': 10, 'n_estimators': 464, 'subsample': 0.563334220385088, 'gamma': 2.58219110212361, 'reg_lambda': 1.8315235668327179}
Best cross-validation accuracy: 0.9436744920453843


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.1588774139080938, device=None,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, feature_types=None, gamma=2.58219110212361,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.12142782379625516,
              max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=12, max_leaves=None,
              min_child_weight=10, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=464, n_jobs=None,
              num_parallel_tree=None, random_state=10, ...)

In [9]:
# Fit the model 
best_model_XGB.fit(X_train_preprocessed, y_train)  # 在整个训练集上训练模型
# Make predictions 
test_preds =best_model_XGB.predict(X_test_preprocessed)
# Create a DataFrame to hold the submission results
output = pd.DataFrame({'id': df_test['id'],
                       'class': test_preds})

# Save the output DataFrame to a CSV file
output.to_csv('submission_XGB.csv', index=False)

output.head()

,id,class
0,140700,0
1,140701,0
2,140702,0
3,140703,1
4,140704,0


# CatBoost

In [10]:
import optuna
from sklearn.model_selection import cross_val_score
from catboost import CatBoostClassifier

# 定义目标函数
def objective(trial):
    # 定义要优化的超参数
    param_grid = {
        'iterations': trial.suggest_int('iterations', 100, 500),
        'depth': trial.suggest_int('depth', 3, 15),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1.0, 10.0),
        'border_count': trial.suggest_int('border_count', 32, 255),
        'bagging_temperature': trial.suggest_float('bagging_temperature', 0.0, 1.0),
    }

    # 初始化模型
    model = CatBoostClassifier(
        **param_grid,
        random_seed=10,
        verbose=0  # 关闭训练过程的输出
    )

    # 交叉验证
    cv_scores = cross_val_score(model, X_train_preprocessed, y_train, cv=5, scoring='accuracy')
    mean_cv_score = cv_scores.mean()

    # 返回目标值（越大越好）
    return mean_cv_score

# 创建 Optuna study
study = optuna.create_study(direction='maximize')  # 最大化准确率
study.optimize(objective, n_trials=50)  # 运行 50 次试验

# 输出最优参数
print("Best parameters:", study.best_params)
print("Best cross-validation accuracy:", study.best_value)

# 使用最优参数初始化模型
best_model_Catboost = CatBoostClassifier(
    **study.best_params,
    random_seed=10,
    verbose=0
)
best_model_Catboost.fit(X_train_preprocessed, y_train)  # 在整个训练集上训练模型


[I 2024-11-07 16:33:32,774] A new study created in memory with name: no-name-b91e8fb4-9933-4970-99fe-f59838a4e459
[I 2024-11-07 16:34:08,265] Trial 0 finished with value: 0.9424973399977234 and parameters: {'iterations': 179, 'depth': 10, 'learning_rate': 0.17420797817471362, 'l2_leaf_reg': 3.594928172191209, 'border_count': 222, 'bagging_temperature': 0.040443159657604366}. Best is trial 0 with value: 0.9424973399977234.
[I 2024-11-07 16:34:19,045] Trial 1 finished with value: 0.9418606385860808 and parameters: {'iterations': 100, 'depth': 3, 'learning_rate': 0.22962032938783342, 'l2_leaf_reg': 9.308993098042873, 'border_count': 208, 'bagging_temperature': 0.766566473668309}. Best is trial 0 with value: 0.9424973399977234.
[I 2024-11-07 16:34:38,576] Trial 2 finished with value: 0.9423788779684312 and parameters: {'iterations': 167, 'depth': 9, 'learning_rate': 0.2562602617233232, 'l2_leaf_reg': 9.918470703333202, 'border_count': 224, 'bagging_temperature': 0.47282602605125335}. Best 

Best parameters: {'iterations': 275, 'depth': 4, 'learning_rate': 0.2558270511554017, 'l2_leaf_reg': 4.888913839629642, 'border_count': 98, 'bagging_temperature': 0.07843948977606202}
Best cross-validation accuracy: 0.9431858587154945


In [11]:
# Fit the model 
best_model_Catboost.fit(X_train_preprocessed, y_train)  # 在整个训练集上训练模型
# Make predictions 
test_preds =best_model_Catboost.predict(X_test_preprocessed)
# Create a DataFrame to hold the submission results
output = pd.DataFrame({'id': df_test['id'],
                       'class': test_preds})

# Save the output DataFrame to a CSV file
output.to_csv('submission_Catboost.csv', index=False)

output.head()

,id,class
0,140700,0
1,140701,0
2,140702,0
3,140703,1
4,140704,0


# LightGBM

In [12]:
import optuna
from sklearn.model_selection import cross_val_score
from lightgbm import LGBMClassifier

# 定义目标函数
def objective(trial):
    # 定义要优化的超参数
    param_grid = {
        'num_leaves': trial.suggest_int('num_leaves', 20, 150),
        'max_depth': trial.suggest_int('max_depth', -1, 15),  # -1 表示不限制深度
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'n_estimators': trial.suggest_int('n_estimators', 100, 500),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 0.0, 10.0),
        'reg_lambda': trial.suggest_float('reg_lambda', 0.0, 10.0),
    }

    # 初始化模型
    model = LGBMClassifier(**param_grid, random_state=10)

    # 交叉验证
    cv_scores = cross_val_score(model, X_train_preprocessed, y_train, cv=5, scoring='accuracy')
    mean_cv_score = cv_scores.mean()

    # 打印当前试验的参数和结果
    print(f"Trial params: {param_grid}")
    print(f"Trial accuracy: {mean_cv_score:.4f}\n")

    # 返回目标值（越大越好）
    return mean_cv_score

# 创建 Optuna study
study = optuna.create_study(direction='maximize')  # 最大化准确率
study.optimize(objective, n_trials=50, show_progress_bar=False)  # 运行 5 次试验

# 输出最优参数
print("Best parameters:", study.best_params)
print("Best cross-validation accuracy:", study.best_value)

# 使用最优参数初始化模型
best_model_LGBM = LGBMClassifier(**study.best_params, random_state=10)
best_model_LGBM.fit(X_train_preprocessed, y_train)  # 在整个训练集上训练模型


[I 2024-11-07 17:24:40,582] A new study created in memory with name: no-name-8fb87ebc-52d9-4f57-b6e5-0455bf90fe97


[LightGBM] [Info] Number of positive: 18660, number of negative: 89397
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.026029 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 639
[LightGBM] [Info] Number of data points in the train set: 108057, number of used features: 18
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.172687 -> initscore=-1.566705
[LightGBM] [Info] Start training from score -1.566705
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] Stoppe

[I 2024-11-07 17:25:01,908] Trial 0 finished with value: 0.9415941249182754 and parameters: {'num_leaves': 39, 'max_depth': -1, 'learning_rate': 0.29294433224948346, 'n_estimators': 480, 'min_child_samples': 58, 'subsample': 0.8605844946335697, 'colsample_bytree': 0.8597539455879686, 'reg_alpha': 7.400688756342555, 'reg_lambda': 6.672181280186916}. Best is trial 0 with value: 0.9415941249182754.


Trial params: {'num_leaves': 39, 'max_depth': -1, 'learning_rate': 0.29294433224948346, 'n_estimators': 480, 'min_child_samples': 58, 'subsample': 0.8605844946335697, 'colsample_bytree': 0.8597539455879686, 'reg_alpha': 7.400688756342555, 'reg_lambda': 6.672181280186916}
Trial accuracy: 0.9416

[LightGBM] [Info] Number of positive: 18660, number of negative: 89397
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.026452 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 639
[LightGBM] [Info] Number of data points in the train set: 108057, number of used features: 18
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.172687 -> initscore=-1.566705
[LightGBM] [Info] Start training from score -1.566705
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, b

[I 2024-11-07 17:25:26,015] Trial 1 finished with value: 0.9426898305868121 and parameters: {'num_leaves': 86, 'max_depth': 6, 'learning_rate': 0.09776996650702148, 'n_estimators': 366, 'min_child_samples': 6, 'subsample': 0.7641699371206854, 'colsample_bytree': 0.7872527150887159, 'reg_alpha': 7.405637236275359, 'reg_lambda': 6.4687857841439325}. Best is trial 1 with value: 0.9426898305868121.


...

[I 2024-11-07 17:41:06,758] Trial 48 finished with value: 0.9401282309577936 and parameters: {'num_leaves': 71, 'max_depth': 0, 'learning_rate': 0.22899824243947586, 'n_estimators': 419, 'min_child_samples': 95, 'subsample': 0.6576176536985552, 'colsample_bytree': 0.6433840293411431, 'reg_alpha': 3.39458548177817, 'reg_lambda': 1.4272728435049062}. Best is trial 43 with value: 0.943630073648858.


Trial params: {'num_leaves': 71, 'max_depth': 0, 'learning_rate': 0.22899824243947586, 'n_estimators': 419, 'min_child_samples': 95, 'subsample': 0.6576176536985552, 'colsample_bytree': 0.6433840293411431, 'reg_alpha': 3.39458548177817, 'reg_lambda': 1.4272728435049062}
Trial accuracy: 0.9401

[LightGBM] [Info] Number of positive: 18660, number of negative: 89397
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.026151 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 639
[LightGBM] [Info] Number of data points in the train set: 108057, number of used features: 18
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.172687 -> initscore=-1.566705
[LightGBM] [Info] Start training from score -1.566705
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, be

[I 2024-11-07 17:41:28,741] Trial 49 finished with value: 0.9422234142657242 and parameters: {'num_leaves': 88, 'max_depth': 4, 'learning_rate': 0.25822779889521585, 'n_estimators': 453, 'min_child_samples': 62, 'subsample': 0.6026109194874055, 'colsample_bytree': 0.7579171044572355, 'reg_alpha': 2.6421947742432375, 'reg_lambda': 5.433056608980936}. Best is trial 43 with value: 0.943630073648858.


Trial params: {'num_leaves': 88, 'max_depth': 4, 'learning_rate': 0.25822779889521585, 'n_estimators': 453, 'min_child_samples': 62, 'subsample': 0.6026109194874055, 'colsample_bytree': 0.7579171044572355, 'reg_alpha': 2.6421947742432375, 'reg_lambda': 5.433056608980936}
Trial accuracy: 0.9422

Best parameters: {'num_leaves': 75, 'max_depth': 3, 'learning_rate': 0.24772427461316152, 'n_estimators': 445, 'min_child_samples': 86, 'subsample': 0.7912543050911118, 'colsample_bytree': 0.7231523758056171, 'reg_alpha': 3.70747272711651, 'reg_lambda': 6.255269886614529}
Best cross-validation accuracy: 0.943630073648858
[LightGBM] [Info] Number of positive: 23325, number of negative: 111747
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.034068 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 654
[LightGBM] [Info] Number of data points in the train set: 135072, number of used features: 18
[LightGBM] [Info] [b

LGBMClassifier(colsample_bytree=0.7231523758056171,
               learning_rate=0.24772427461316152, max_depth=3,
               min_child_samples=86, n_estimators=445, num_leaves=75,
               random_state=10, reg_alpha=3.70747272711651,
               reg_lambda=6.255269886614529, subsample=0.7912543050911118)

In [13]:
# Fit the model 
best_model_LGBM.fit(X_train_preprocessed, y_train)

# Make predictions 
test_preds = best_model_LGBM.predict(X_test_preprocessed)
# Create a DataFrame to hold the submission results
output = pd.DataFrame({'id': df_test['id'],
                       'class': test_preds})

# Save the output DataFrame to a CSV file
output.to_csv('submission_LGBM.csv', index=False)
output.head()

[LightGBM] [Info] Number of positive: 23325, number of negative: 111747
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.033718 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 654
[LightGBM] [Info] Number of data points in the train set: 135072, number of used features: 18
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.172686 -> initscore=-1.566712
[LightGBM] [Info] Start training from score -1.566712
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive ga

,id,class
0,140700,0
1,140701,0
2,140702,0
3,140703,1
4,140704,0


# ENSEMBLE

In [14]:
import numpy as np

# 假设 X_test_preprocessed 是测试集的特征
# 获取每个模型的预测概率
probs_XGB = best_model_XGB.predict_proba(X_test_preprocessed)
probs_CatBoost = best_model_Catboost.predict_proba(X_test_preprocessed)
probs_LGBM = best_model_LGBM.predict_proba(X_test_preprocessed)

# 设置每个模型的权重（这里给定相同的权重 1，表示平等融合）
w1, w2, w3 = 1, 1, 1

# 加权平均融合概率
final_probs = (w1 * probs_XGB + w2 * probs_CatBoost + w3 * probs_LGBM) / (w1 + w2 + w3)

# 得到最终预测类别
final_preds = np.argmax(final_probs, axis=1)

# 创建提交文件
output = pd.DataFrame({'id': df_test['id'], 'class': final_preds})
output.to_csv('submission_ensemble3.csv', index=False)

# 显示提交文件的前几行
print(output.head())


       id  class
0  140700      0
1  140701      0
2  140702      0
3  140703      1
4  140704      0


thanks